In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
intake_df = pd.read_csv('Austin Animal Center Intakes.csv')
outtake_df = pd.read_csv('Austin Animal Center Outcomes.csv')

In [4]:
intake_df.describe()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
count,157627,112208,157627,157627,157627,157627,157627,157627,157625,157626,157627,157627
unique,141420,27590,109311,123,64704,6,20,5,5,54,2887,635
top,A721033,Max,09/23/2016 12:00:00 PM,June 2015,Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,680,64,2189,28977,108088,134680,87191,53011,25931,33285,16428


In [5]:
outtake_df.describe()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
count,157566,112276,157566,157566,157566,157536,72548,157566,157564,157564,157566,157566
unique,141347,27503,131291,123,8153,10,26,5,5,54,2889,637
top,A721033,Max,04/18/2016 12:00:00 AM,Jun 2019,05/01/2016,Adoption,Partner,Dog,Neutered Male,1 year,Domestic Shorthair Mix,Black/White
freq,33,678,39,2244,120,74468,37451,87091,55185,26334,33626,16429


The difference between count of Animal ID and unique values of Animal ID in the intake file suggest that Animal ID alone isn't a sufficent way to identify animals for analysing intakes.

In [6]:
print(f"All animal_id values in intake file are unique: {intake_df['Animal ID'].is_unique} All animal_id values in outtake file are unique: {outtake_df['Animal ID'].is_unique}")

All animal_id values in intake file are unique: False All animal_id values in outtake file are unique: False


In [ ]:
duplicate_intake_animals = intake_df.loc[intake_df['Animal ID'].duplicated(keep=False), 'Animal ID'].unique()
duplicate_rows = intake_df[intake_df['Animal ID'].isin(duplicate_intake_animals)]
print(duplicate_rows.sort_values(by='Animal ID'))

In [ ]:
duplicate_outtake_animals = outtake_df.loc[outtake_df['Animal ID'].duplicated(keep=False), 'Animal ID'].unique()
duplicate_rows_outtake = outtake_df[outtake_df['Animal ID'].isin(duplicate_outtake_animals)]
print(duplicate_rows_outtake.sort_values(by='Animal ID'))

Looking at Animal ID A893746 we see they're outtaked twice with the same reason "Adoption." Is there a intake between those two adoption times, signifying a return? 

Watson adoption 1: 5/6/23
Watson owner surrender: 5/25/23
Watson adoption 2: 9/9/23

In [ ]:
print(intake_df[intake_df['Animal ID'] == 'A893746'])

Let's look at Animal ID A892023 (Willard) who was marked for adoption on both 11/13/23 and 11/26/23. Is there intake activity in between these dates?

Willard adoption 1: 11/13/23
Willard owner surrender: 11/15/23
Willard adoption 2: 11/26/23

In [ ]:
print(intake_df[intake_df['Animal ID'] == 'A892023'])

How many "true duplicates" are there for inttake or outtake records?

In [11]:
intake_duplicates = intake_df.duplicated().sum()
outtake_duplicates = outtake_df.duplicated().sum()

print(f"Intake duplicates {intake_duplicates} Outtake duplicates {outtake_duplicates}")

Intake duplicates 29 Outtake duplicates 23


Let's go ahead and remove all "true duplicates" from our dataframe

In [12]:
intake_df = intake_df.drop_duplicates()
outtake_df = outtake_df.drop_duplicates()

In [13]:
#Sanity check
intake_duplicates = intake_df.duplicated().sum()
outtake_duplicates = outtake_df.duplicated().sum()

print(f"Intake duplicates {intake_duplicates} Outtake duplicates {outtake_duplicates}")

Intake duplicates 0 Outtake duplicates 0


In [ ]:
print(intake_df.isnull().sum())
print(outtake_df.isnull().sum())

Looking at the dataframes, month_year is formatted differently across the two files:

Outtake has a truncated month signature, while intakes print the full month. This can cause issues when cross comparing the two by time.

In [15]:
print(f"Intake MonthYear is formatted like {intake_df.sample(n=1)['MonthYear'].values[0]} while outtake MonthYear is formatted like {outtake_df.sample(n=1)['MonthYear'].values[0]}")

Intake MonthYear is formatted like February 2019 while outtake MonthYear is formatted like Jan 2016


To resolve this, we will convert the intake MonthYear value to match the outtake MonthYear value

In [16]:
intake_df['MonthYear'] = pd.to_datetime(intake_df['MonthYear'], format='%B %Y')
intake_df['MonthYear'] = intake_df['MonthYear'].dt.strftime('%b %Y')

In [ ]:
print(intake_df['MonthYear'])

Let's check for any variations in text fields, such as misspellings or different cases

In [ ]:
text_based_intake_columns = ['Intake Type', 'Intake Condition', 'Animal Type', 'Sex upon Intake', 'Age upon Intake', 'Breed', 'Color']
text_based_outtake_columns = ['Outcome Type', 'Outcome Subtype', 'Animal Type', 'Sex upon Outcome', 'Age upon Outcome', 'Breed', 'Color']

print("INTAKES:")
for column_name in text_based_intake_columns:
    print(intake_df[column_name].value_counts())

print("OUTTAKES:")
for column_name in text_based_outtake_columns:
    print(outtake_df[column_name].value_counts())


Through observation, we notice no two distinct categories for the same thing (i.e. a Intake Condition category of "Sick" and "Illness"). However, one thing to note is that the Age upon Intake and Age upon Outtake do not have a standardized format. Some go by months, days, years. For data analysis, we need to standardize these into one intger.

For this project, we will be standardizing all of the ages to be number of months.

For the 'day' case, we will assume 1 month = 30 days for convenience

In [19]:
def age_to_months(given_age):
    if pd.isna(given_age):
         return np.nan 
    # This will strip it into an array of [Number, Time Unit]
    # For example, "1 year" -> ["1", "year"]
    given_age = given_age.lower().strip().split()
    if 'year' in given_age[1]:
         return int(given_age[0]) * 12
    elif 'month' in given_age[1]:
         return int(given_age[0])
    elif 'week' in given_age[1]:
         return int(given_age[0]) / 4
    elif 'day' in given_age[1]:
         return int(given_age[0]) / 30
    else:
         return np.nan
         

In [ ]:
for age in intake_df['Age upon Intake']:
    converted = age_to_months(age)
    print(f"Original: {age} => Converted: {converted}")

In [21]:
intake_df['Age upon Intake'] = intake_df['Age upon Intake'].apply(age_to_months)
outtake_df['Age upon Outcome'] = outtake_df['Age upon Outcome'].apply(age_to_months)

We notice some negative age values.

In [ ]:
print(intake_df[intake_df['Age upon Intake'] < 0])

In [ ]:
print(outtake_df[outtake_df['Age upon Outcome'] < 0])

For this project, let's set values < 0 to NaN

In [24]:
intake_df.loc[intake_df['Age upon Intake'] < 0, 'Age upon Intake'] = np.nan
outtake_df.loc[outtake_df['Age upon Outcome'] < 0, 'Age upon Outcome'] = np.nan

In [ ]:
print(intake_df)

In [ ]:
print(outtake_df)

In [30]:
intake_df.to_csv('processed_animal_intake.csv', index=False)
outtake_df.to_csv('processed_animal_outtake.csv', index=False)